# Lanchain : Summarization

## Setup

### Libraries

In [1]:
from typing import List, Optional, Union

import arxiv
import numpy as np
import pandas as pd
import plotly.express as px
import torch
import umap
import yaml
from langchain import HuggingFaceHub, LLMChain, PromptTemplate
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.schema.document import Document
from langchain.text_splitter import CharacterTextSplitter
from rich import print
from sklearn.cluster import DBSCAN
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    BigBirdPegasusForConditionalGeneration,
    PegasusTokenizerFast,
)

### Check GPU availability

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
if device == "cuda:0":
    print(torch.cuda.get_device_properties(device))
else:
    print(f"No cuda device found; running on {device}")

_CudaDeviceProperties(name='NVIDIA GeForce RTX 3090 Ti', major=8, minor=6, total_memory=24563MB, 
multi_processor_count=84)

## Load secrets

In [3]:
with open("../conf/local/hf_secrets.yml", "r") as f:
    hf_secrets = yaml.load(f, Loader=yaml.SafeLoader)

## Load data

In [4]:
# run search query on arxiv
search = arxiv.Search(
    query="ti:LLM OR (ti:LARGE AND ti:LANGUAGE AND ti:MODEL)",
    max_results=100,
    sort_by=arxiv.SortCriterion.SubmittedDate,
)

# display some results
for result in list(search.results())[:2]:
    print(f"--- {result.title} [{result.published}] ---")
    print(result.summary)

--- Enhancing Network Management Using Code Generated by Large Language Models [2023-08-11 17:49:15+00:00] ---

Analyzing network topologies and communication graphs plays a crucial role in
contemporary network management. However, the absence of a cohesive approach
leads to a challenging learning curve, heightened errors, and inefficiencies.
In this paper, we introduce a novel approach to facilitate a
natural-language-based network management experience, utilizing large language
models (LLMs) to generate task-specific code from natural language queries.
This method tackles the challenges of explainability, scalability, and privacy
by allowing network operators to inspect the generated code, eliminating the
need to share network data with LLMs, and concentrating on application-specific
requests combined with general program synthesis techniques. We design and
evaluate a prototype system using benchmark applications, showcasing high
accuracy, cost-effectiveness, and the potential for further enhancements using
complementary program synthesis techniques.

--- A Large Language Model Enhanced Conversational Recommender System [2023-08-11 16:30:44+00:00] ---

Conversational recommender systems (CRSs) aim to recommend high-quality items
to users through a dialogue interface. It usually contains multiple sub-tasks,
such as user preference elicitation, recommendation, explanation, and item
information search. To develop effective CRSs, there are some challenges: 1)
how to properly manage sub-tasks; 2) how to effectively solve different
sub-tasks; and 3) how to correctly generate responses that interact with users.
Recently, Large Language Models (LLMs) have exhibited an unprecedented ability
to reason and generate, presenting a new opportunity to develop more powerful
CRSs. In this work, we propose a new LLM-based CRS, referred to as LLMCRS, to
address the above challenges. For sub-task management, we leverage the
reasoning ability of LLM to effectively manage sub-task. For sub-task solving,
we collaborate LLM with expert models of different sub-tasks to achieve the
enhanced performance. For response generation, we utilize the generation
ability of LLM as a language interface to better interact with users.
Specifically, LLMCRS divides the workflow into four stages: sub-task detection,
model matching, sub-task execution, and response generation. LLMCRS also
designs schema-based instruction, demonstration-based instruction, dynamic
sub-task and model matching, and summary-based generation to instruct LLM to
generate desired results in the workflow. Finally, to adapt LLM to
conversational recommendations, we also propose to fine-tune LLM with
reinforcement learning from CRSs performance feedback, referred to as RLPF.
Experimental results on benchmark datasets show that LLMCRS with RLPF
outperforms the existing methods.

## Cluster Articles

Interesting models to try:
* [google/bigbird-pegasus-large-arxiv](https://huggingface.co/google/bigbird-pegasus-large-arxiv)
* [allenai/led-large-16384-arxiv](https://huggingface.co/allenai/led-large-16384-arxiv)
* [google/pegasus-arxiv](https://huggingface.co/google/pegasus-arxiv)
* [facebook/bart-large-cnn](https://huggingface.co/facebook/bart-large-cnn) (very interesting summarization results from that one)

Interesting links for embeddings & langchain:
* [HuggingFaceBgeEmbeddings](https://api.python.langchain.com/en/latest/embeddings/langchain.embeddings.huggingface.HuggingFaceBgeEmbeddings.html?highlight=device)
* [HuggingFaceEmbeddings](https://api.python.langchain.com/en/latest/embeddings/langchain.embeddings.huggingface.HuggingFaceEmbeddings.html?highlight=device)
* [HuggingFaceInstructEmbeddings](https://api.python.langchain.com/en/latest/embeddings/langchain.embeddings.huggingface.HuggingFaceInstructEmbeddings.html?highlight=device)

In [5]:
# Load model
model_name = "google/bigbird-pegasus-large-arxiv"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [6]:
# get abstract
result = next(search.results())
summary = result.summary

# feed-forward pass
inputs = tokenizer([summary], max_length=4096, return_tensors="pt", truncation=True)
outputs = model(**inputs, return_dict=True)

# Generate Summary
# summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=15)
# model_summary = tokenizer.batch_decode(summary_ids,
# skip_special_tokens=True,
# clean_up_tokenization_spaces=False)[0]
# print(f"model_summary: '{model_summary}'")

# Retrieve embedding
cls_token_embed = outputs.encoder_last_hidden_state[:, 0, :]

Attention type 'block_sparse' is not possible if sequence_length: 158 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


In [7]:
def embed_pegasus(
    input_text: Union[str, List[str]],
    model: BigBirdPegasusForConditionalGeneration,
    tokenizer: PegasusTokenizerFast,
    batch_size: Optional[int] = None,
    device: str = "cpu",
) -> Union[np.ndarray, torch.Tensor]:
    model = model.to(device)

    dim = model.config.hidden_size

    if isinstance(input_text, str):
        inputs = tokenizer(
            [input_text],
            max_length=4096,
            return_tensors="pt",
            truncation=True,
            padding=True,
        )
        outputs = model(**inputs.to(device), return_dict=True)
        cls_token_embeds = outputs.encoder_last_hidden_state[:, 0, :]

        del outputs
        del inputs
        torch.cuda.empty_cache()

    else:
        if batch_size is not None:
            indices = list(range(0, len(input_text), batch_size)) + [None]
            cls_token_embeds = torch.empty((0, dim), device=device)

            for i in range(len(indices) - 1):
                input_text_i = input_text[indices[i] : indices[i + 1]]
                inputs_i = tokenizer(
                    input_text_i,
                    max_length=4096,
                    return_tensors="pt",
                    truncation=True,
                    padding=True,
                )
                outputs_i = model(**inputs_i.to(device), return_dict=True)
                cls_token_embeds_i = outputs_i.encoder_last_hidden_state[:, 0, :]
                cls_token_embeds = torch.cat((cls_token_embeds, cls_token_embeds_i), dim=0)

                del outputs_i
                del inputs_i
                torch.cuda.empty_cache()

        else:
            inputs = tokenizer(input_text, max_length=4096, return_tensors="pt", truncation=True)
            outputs = model(**inputs.to(device), return_dict=True)
            cls_token_embeds = outputs.encoder_last_hidden_state[:, 0, :]

            del outputs
            del inputs
            torch.cuda.empty_cache()

    return cls_token_embeds

In [8]:
# retrieve abstracts & metadata
abstracts = [result.summary for result in search.results()]
titles = [result.title for result in search.results()]

# embed abstracts
with torch.no_grad():
    embeddings = embed_pegasus(
        input_text=abstracts,
        model=model,
        tokenizer=tokenizer,
        batch_size=20,
        device=device,
    )

# convert to numpy
if device == "cpu":
    embeddings = embeddings.numpy()
else:
    embeddings = embeddings.detach().cpu().numpy()

# reduce dimension
reducer = umap.UMAP(n_components=2, n_neighbors=5, metric="cosine", min_dist=0.1)
umap_coords = reducer.fit_transform(embeddings)

In [9]:
# store results in a dataframe
df = pd.DataFrame(data=umap_coords, columns=[f"umap_{i}" for i in range(umap_coords.shape[1])])
df["title"] = titles

# display
df.head()

,umap_0,umap_1,title
0,6.530040,-2.340559,Enhancing Network Management Using Code Genera...
1,-2.170251,4.997292,A Large Language Model Enhanced Conversational...
2,4.014503,9.155603,Improving Zero-Shot Text Matching for Financia...
3,5.358764,9.537405,Assessing Student Errors in Experimentation Us...
4,3.699078,9.688238,Learning to Guide Human Experts via Personaliz...


In [10]:
# viz
fig = px.scatter(df, x="umap_0", y="umap_1", hover_data="title")
fig.show()

In [11]:
# clustering with (H)DBSCAN

clusterer = DBSCAN(eps=0.8)
clusterer.fit(df[[col for col in df.columns if "umap" in col]])
df["cluster"] = clusterer.labels_

# viz
fig = px.scatter(df, x="umap_0", y="umap_1", hover_data="title", color="cluster")
fig.show()

## Summarize clusters of articles

Interesting links:
* [https://python.langchain.com/docs/integrations/retrievers/arxiv](https://python.langchain.com/docs/integrations/retrievers/arxiv)
* [https://python.langchain.com/docs/additional_resources/tutorials](https://python.langchain.com/docs/additional_resources/tutorials)
* [https://python.langchain.com/docs/use_cases/summarization](https://python.langchain.com/docs/use_cases/summarization)
* [https://huggingface.co/facebook/bart-large-cnn](https://huggingface.co/facebook/bart-large-cnn)
* [https://python.langchain.com/docs/integrations/llms/openllm](https://python.langchain.com/docs/integrations/llms/openllm)

### Load model

In [12]:
# load model
# See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options
repo_id = "facebook/bart-large-cnn"
llm = HuggingFaceHub(
    repo_id=repo_id,
    model_kwargs={
        "temperature": 0.15,
        "max_length": 100,
        # "device": device,
    },
    huggingfacehub_api_token=hf_secrets["hf_hub_token"],
)


# question = "Who won the FIFA World Cup in the year 1994? "
# template = """
# Question: {question}

# Answer:
# """

# prompt = PromptTemplate(template=template, input_variables=["question"])

# llm_chain = LLMChain(prompt=prompt, llm=llm)
# print(llm_chain.run(question))

### Option 1: Stuff

### Option 2: Map reduce

In [19]:
# build docs

cluster_idx = 0
docs = [
    Document(
        page_content=abstracts[i],
        metadata={"title": titles[i]},
    )
    for i in range(len(titles))
    if df.loc[df["title"] == titles[i], "cluster"].values[0] == cluster_idx
]

In [20]:
print(docs[:2])

[
    Document(
        page_content='Analyzing network topologies and communication graphs plays a crucial role in\ncontemporary 
network management. However, the absence of a cohesive approach\nleads to a challenging learning curve, heightened 
errors, and inefficiencies.\nIn this paper, we introduce a novel approach to facilitate a\nnatural-language-based 
network management experience, utilizing large language\nmodels (LLMs) to generate task-specific code from natural 
language queries.\nThis method tackles the challenges of explainability, scalability, and privacy\nby allowing 
network operators to inspect the generated code, eliminating the\nneed to share network data with LLMs, and 
concentrating on application-specific\nrequests combined with general program synthesis techniques. We design 
and\nevaluate a prototype system using benchmark applications, showcasing high\naccuracy, cost-effectiveness, and 
the potential for further enhancements using\ncomplementary program synthesis techniques.',
        metadata={'title': 'Enhancing Network Management Using Code Generated by Large Language Models'}
    ),
    Document(
        page_content="Large Language Models (LLMs) could enhance access to the legal system.\nHowever, empirical 
research on their effectiveness in conducting legal tasks is\nscant. We study securities cases involving 
cryptocurrencies as one of numerous\ncontexts where AI could support the legal process, studying LLMs' 
legal\nreasoning and drafting capabilities. We examine whether a) an LLM can\naccurately determine which laws are 
potentially being violated from a fact\npattern, and b) whether there is a difference in juror decision-making 
based on\ncomplaints written by a lawyer compared to an LLM. We feed fact patterns from\nreal-life cases to GPT-3.5
and evaluate its ability to determine correct\npotential violations from the scenario and exclude spurious 
violations. Second,\nwe had mock jurors assess complaints written by the LLM and lawyers. GPT-3.5's\nlegal 
reasoning skills proved weak, though we expect improvement in future\nmodels, particularly given the violations it 
suggested tended to be correct (it\nmerely missed additional, correct violations). GPT-3.5 performed better 
at\nlegal drafting, and jurors' decisions were not statistically significantly\nassociated with the author of the 
document upon which they based their\ndecisions. Because LLMs cannot satisfactorily conduct legal reasoning 
tasks,\nthey would be unable to replace lawyers at this stage. However, their drafting\nskills (though, perhaps, 
still inferior to lawyers), could provide access to\njustice for more individuals by reducing the cost of legal 
services. Our\nresearch is the first to systematically study LLMs' legal drafting and\nreasoning capabilities in 
litigation, as well as in securities law and\ncryptocurrency-related misconduct.",
        metadata={
            'title': 'Large Language Models in Cryptocurrency Securities Cases: Can ChatGPT Replace Lawyers?'
        }
    )
]

In [21]:
# Map
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

In [22]:
# Reduce
reduce_template = """The following is set of summaries:
{doc_summaries}
Take these and distill it into a final, consolidated summary of the main themes.
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(llm_chain=reduce_chain, document_variable_name="doc_summaries")

# Combines and iteravely reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

In [23]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)

In [24]:
print(map_reduce_chain.run(split_docs))

### Option 3: Refine